In [20]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib as plt
import json
from pandas.io.json import json_normalize
import ijson
import sqlite3

In [21]:
data_demo = pd.read_csv("Demographic_Statistics_By_Zip_Code.csv")
data_demo = data_demo.rename(columns={ data_demo.columns[0]: "zip_code" })
data_demo.head()

,zip_code,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,44,100,20,0.45,24,0.55,0,0,44,100
1,10002,35,19,0.54,16,0.46,0,0,35,100,...,35,100,2,0.06,33,0.94,0,0,35,100
2,10003,1,1,1.00,0,0.00,0,0,1,100,...,1,100,0,0.00,1,1.00,0,0,1,100
3,10004,0,0,0.00,0,0.00,0,0,0,0,...,0,0,0,0.00,0,0.00,0,0,0,0
4,10005,2,2,1.00,0,0.00,0,0,2,100,...,2,100,0,0.00,2,1.00,0,0,2,100


In [22]:
#with open('graffit.json') as data_file:
#    data2 = json.load(data_file)
#print(json.dumps(data2, indent=2, sort_keys=True))
#for key, value in data2.items():
#    print(key,value)
filename = "graffit.json"
with open(filename, 'r') as f:
    objects = ijson.items(f, 'meta.view.columns.item')
    columns = list(objects)
column_names = [col["fieldName"] for col in columns]
column_names

[':sid',
 ':id',
 ':position',
 ':created_at',
 ':created_meta',
 ':updated_at',
 ':updated_meta',
 ':meta',
 'incident_address',
 'borough',
 'community_board',
 'police_precinct',
 'city_council_district',
 'bbl',
 'created_date',
 'status',
 'resolution_action',
 'closed_date',
 'x_coordinate',
 'y_coordinate',
 'latitude',
 'longitude',
 'zip_code',
 'census_tract',
 'bin',
 'nta',
 'location']

In [23]:
good_columns = [
 'incident_address',
 'borough',
 'community_board',
 'police_precinct',
 'city_council_district',
 'bbl',
 'created_date',
 'status',
 'resolution_action',
 'closed_date',
 'x_coordinate',
 'y_coordinate',
 'latitude',
 'longitude',
 'zip_code',
 'census_tract',
 'bin',
 'nta',
 'location'
]

data = []
with open(filename, 'r') as f:
    objects = ijson.items(f, 'data.item')
    for row in objects:
        selected_row = []
        for item in good_columns:
            selected_row.append(row[column_names.index(item)])
        data.append(selected_row)
data

[['43 WEST   65 STREET',
  'MANHATTAN',
  '07 MANHATTAN',
  'Precinct 20',
  '06',
  '1011180005',
  '2018-03-11T00:00:00',
  'Open',
  'Notice of Intent to Clean and Forever graffiti free form sent',
  None,
  '989655',
  '220659',
  '40.772333',
  '-73.980486',
  '10023',
  ' 149  ',
  '1028160',
  'Lincoln Square',
  [None, '40.772333', '-73.980486', None, False]],
 ['1700 DITMAS AVENUE',
  'BROOKLYN',
  '14 BROOKLYN',
  'Precinct 70',
  '40',
  '3052030001',
  '2018-03-10T00:00:00',
  'Open',
  'Notice of Intent to Clean and Forever graffiti free form sent',
  None,
  '994870',
  '171715',
  '40.637989',
  '-73.961735',
  '11226',
  ' 518  ',
  '3120202',
  'Flatbush',
  [None, '40.637989', '-73.961735', None, False]],
 ['504 EAST   11 STREET',
  'MANHATTAN',
  '03 MANHATTAN',
  'Precinct 9',
  '02',
  '1004040006',
  '2018-03-09T00:00:00',
  'Open',
  'Notice of Intent to Clean and Forever graffiti free form sent',
  None,
  '989410',
  '204604',
  '40.728266',
  '-73.981383',
  '

In [24]:
graffiti_df = pd.DataFrame(data, columns=good_columns)
#graffiti_df = graffiti_df.set_index('zip_code')
#graffiti_df = graffiti_df['zip_code'].astype(str)
#graffiti_df = graffiti_df[~graffiti_df.zip_code.str.contains("None")]
graffiti_df

,incident_address,borough,community_board,police_precinct,city_council_district,bbl,created_date,status,resolution_action,closed_date,x_coordinate,y_coordinate,latitude,longitude,zip_code,census_tract,bin,nta,location
0,43 WEST 65 STREET,MANHATTAN,07 MANHATTAN,Precinct 20,06,1011180005,2018-03-11T00:00:00,Open,Notice of Intent to Clean and Forever graffiti...,None,989655,220659,40.772333,-73.980486,10023,149,1028160,Lincoln Square,"[None, 40.772333, -73.980486, None, False]"
1,1700 DITMAS AVENUE,BROOKLYN,14 BROOKLYN,Precinct 70,40,3052030001,2018-03-10T00:00:00,Open,Notice of Intent to Clean and Forever graffiti...,None,994870,171715,40.637989,-73.961735,11226,518,3120202,Flatbush,"[None, 40.637989, -73.961735, None, False]"
2,504 EAST 11 STREET,MANHATTAN,03 MANHATTAN,Precinct 9,02,1004040006,2018-03-09T00:00:00,Open,Notice of Intent to Clean and Forever graffiti...,None,989410,204604,40.728266,-73.981383,10009,34,1076974,East Village,"[None, 40.728266, -73.981383, None, False]"
3,494 MADISON STREET,BROOKLYN,03 BROOKLYN,Precinct 81,36,3016440006,2018-03-07T00:00:00,Open,Notice of Intent to Clean and Forever graffiti...,None,1001445,189269,40.686160,-73.938000,11221,277,3045053,Stuyvesant Heights,"[None, 40.68616, -73.938, None, False]"
4,1770 PITKIN AVENUE,BROOKLYN,16 BROOKLYN,Precinct 73,41,3035250113,2018-03-06T00:00:00,Open,Notice of Intent to Clean and Forever graffiti...,None,1009832,183432,40.670119,-73.907781,11212,906,3348958,Brownsville,"[None, 40.670119, -73.907781, None, False]"
5,49-09 NORTHERN BOULEVARD,QUEENS,01 QUEENS,Precinct 114,26,4007350005,2018-03-02T00:00:00,Open,Site to be cleaned.,None,1008260,213788,40.753443,-73.913340,11101,163,4013651,Astoria,"[None, 40.753443, -73.91334, None, False]"
6,5405 6 AVENUE,BROOKLYN,07 BROOKLYN,Precinct 72,38,3008250008,2018-03-01T00:00:00,Open,Site to be cleaned.,None,981324,172967,40.641431,-74.010543,11220,100,3014720,Sunset Park East,"[None, 40.641431, -74.010543, None, False]"
7,137 FOUNTAIN AVENUE,BROOKLYN,05 BROOKLYN,Precinct 75,37,3041910011,2018-02-27T00:00:00,Open,Notice of Intent to Clean and Forever graffiti...,None,1018718,185888,40.676830,-73.875736,11208,1196,3389426,East New York,"[None, 40.67683, -73.875736, None, False]"
8,110-54 QUEENS BOULEVARD,QUEENS,06 QUEENS,Precinct 112,29,4032927501,2018-02-22T00:00:00,Open,Site downloaded for cleaning,None,1028821,201387,40.719326,-73.839210,11375,76901,4078799,Forest Hills,"[None, 40.719326, -73.83921, None, False]"
9,201 11 AVENUE,MANHATTAN,04 MANHATTAN,Precinct 10,03,1006700001,2018-02-22T00:00:00,Open,Graffiti Reported,None,982457,212395,40.749652,-74.006471,10001,99,1012266,Hudson Yards-Chelsea-Flatiron-Union Square,"[None, 40.749652, -74.006471, None, False]"


In [25]:
def parse_float(x):
    try:
        x = float(x)
    except Exception:
        x = 0
    return x
graffiti_df["zip_code"] = graffiti_df["zip_code"].apply(parse_float)



In [32]:
graffiti_df['incident_address']= graffiti_df['incident_address'].apply(parse_float)
graffiti_df['borough']= graffiti_df['borough'].apply(parse_float)
graffiti_df['police_precinct']= graffiti_df['police_precinct'].apply(parse_float)
graffiti_df['city_council_district']= graffiti_df['city_council_district'].apply(parse_float)
graffiti_df['bbl']= graffiti_df['bbl'].apply(parse_float)
graffiti_df['created_date']= graffiti_df['created_date'].apply(parse_float)
graffiti_df['status']= graffiti_df['status'].apply(parse_float)
graffiti_df['resolution_action']= graffiti_df['resolution_action'].apply(parse_float)
graffiti_df['closed_date']= graffiti_df['closed_date'].apply(parse_float)
graffiti_df['resolution_action']= graffiti_df['resolution_action'].apply(parse_float)
graffiti_df['x_coordinate']= graffiti_df['x_coordinate'].apply(parse_float)
graffiti_df['y_coordinate']= graffiti_df['y_coordinate'].apply(parse_float)
graffiti_df['latitude']= graffiti_df['latitude'].apply(parse_float)
graffiti_df['longitude']= graffiti_df['longitude'].apply(parse_float)
graffiti_df['census_tract']= graffiti_df['census_tract'].apply(parse_float)
graffiti_df['bin']= graffiti_df['bin'].apply(parse_float)
graffiti_df['nta']= graffiti_df['nta'].apply(parse_float)
graffiti_df['location']= graffiti_df['location'].apply(parse_float)

#anagramsdf.to_sql('anagramsdf',con=conn,if_exists='append',index=False)

In [33]:
graffiti_df.dtypes


incident_address           int64
borough                    int64
community_board           object
police_precinct            int64
city_council_district    float64
bbl                      float64
created_date               int64
status                     int64
resolution_action        float64
closed_date                int64
x_coordinate             float64
y_coordinate             float64
latitude                 float64
longitude                float64
zip_code                 float64
census_tract             float64
bin                      float64
nta                        int64
location                   int64
dtype: object

In [34]:
#df_merge = pd.concat([data_demo, graffiti_df], sort=False)
#frames = [data_demo, graffiti_df]
#result = pd.concat(frames)
#result
#result = data_demo.set_index('zip_code').join(graffiti_df.set_index('zip_code'))
#result
result = data_demo.merge(graffiti_df, on='zip_code', how='inner', suffixes=('_1', '_2'))
result

,zip_code,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,resolution_action,closed_date,x_coordinate,y_coordinate,latitude,longitude,census_tract,bin,nta,location
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,982457.0,212395.0,40.749652,-74.006471,99.0,1012266.0,0,0
1,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,986732.0,213269.0,40.752051,-73.991042,109.0,1014427.0,0,0
2,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,983747.0,212229.0,40.749196,-74.001815,97.0,1012827.0,0,0
3,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,988137.0,211562.0,40.747365,-73.985972,76.0,1083631.0,0,0
4,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,988376.0,212356.0,40.749544,-73.985109,84.0,1015894.0,0,0
5,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,987649.0,211532.0,40.747283,-73.987733,76.0,1078654.0,0,0
6,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,988361.0,212364.0,40.749566,-73.985163,84.0,1015893.0,0,0
7,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,987420.0,212886.0,40.750999,-73.988559,109.0,1015225.0,0,0
8,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,984070.0,211588.0,40.747437,-74.000650,93.0,1012802.0,0,0
9,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,985562.0,213600.0,40.752959,-73.995265,111.0,1013555.0,0,0


In [35]:
import sqlite3

conn = sqlite3.connect('test.sqlite')

result.to_sql('tab', conn, if_exists='replace', index=False)

pd.read_sql('select * from tab', conn)

,zip_code,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,resolution_action,closed_date,x_coordinate,y_coordinate,latitude,longitude,census_tract,bin,nta,location
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,982457.0,212395.0,40.749652,-74.006471,99.0,1012266.0,0,0
1,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,986732.0,213269.0,40.752051,-73.991042,109.0,1014427.0,0,0
2,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,983747.0,212229.0,40.749196,-74.001815,97.0,1012827.0,0,0
3,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,988137.0,211562.0,40.747365,-73.985972,76.0,1083631.0,0,0
4,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,988376.0,212356.0,40.749544,-73.985109,84.0,1015894.0,0,0
5,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,987649.0,211532.0,40.747283,-73.987733,76.0,1078654.0,0,0
6,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,988361.0,212364.0,40.749566,-73.985163,84.0,1015893.0,0,0
7,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,987420.0,212886.0,40.750999,-73.988559,109.0,1015225.0,0,0
8,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,984070.0,211588.0,40.747437,-74.000650,93.0,1012802.0,0,0
9,10001,44,22,0.50,22,0.50,0,0,44,100,...,0.0,0,985562.0,213600.0,40.752959,-73.995265,111.0,1013555.0,0,0


In [86]:
export_csv = result.to_csv ('export_dataframe.csv', index = None, header=True)